
# Assignment 3

This is a template notebook for Assignment 3.


## Install dependencies and initialization

In [ ]:
# install dependencies: 
!pip install pyyaml==5.1 pycocotools>=2.0.1
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html

In [2]:
!pwd # shows current directory
!ls  # shows all files in this directory
!nvidia-smi # shows the specs and the current status of the allocated GPU

/content
'=2.0.1'   data   drive   output   sample_data
Tue Nov  3 05:42:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    23W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                

In [3]:
# import some common libraries
from google.colab.patches import cv2_imshow
from sklearn.metrics import jaccard_score
from PIL import Image, ImageDraw
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import datetime
import random
import json
import cv2
import csv
import os

# import some common pytorch utilities
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch

# import some common detectron2 utilities
import detectron2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
from detectron2.data import build_detection_test_loader
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
import matplotlib.pyplot as plt
setup_logger()

<Logger detectron2 (DEBUG)>

In [4]:
# Make sure that GPU is available for your notebook. 
# Otherwise, you need to update the settungs in Runtime -> Change runtime type -> Hardware accelerator
torch.cuda.is_available()

True

In [5]:
# You need to mount your google drive in order to load the data:
from google.colab import drive
drive.mount('/content/drive')
# Put all the corresponding data files in a data folder and put the data folder in a same directory with this notebook.
# Also create an output directory for your files such as the trained models and the output images.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
 # ! unzip "/content/drive/My Drive/Colab Notebooks/03-cnn-detection-segmentation/data/plane_dataset.zip" > /dev/null

In [6]:
# Define the location of current directory, which should contain data/train, data/test, and data/train.json.
# TODO: approx 1 line
Curr='data'
BASE_DIR = '/content/drive/My Drive/Colab Notebooks/03-cnn-detection-segmentation'
OUTPUT_DIR = '{}/output'.format(BASE_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

## Part 1: Object Detection

### Data Loader

In [7]:
'''
# This function should return a list of data samples in which each sample is a dictionary. 
# Make sure to select the correct bbox_mode for the data
# For the test data, you only have access to the images, therefore, the annotations should be empty.
# Other values could be obtained from the image files.
# TODO: approx 35 lines
'''

# refer from: https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5#scrollTo=PIbAM2pv-urF
def get_detection_data(set_name):
  data_dirs = '{}/data'.format(BASE_DIR)
  dataset = []

  # load train.json 
  if 'train' in set_name:
    
    json_file=os.path.join("data","train.json")
    with open(json_file) as f:
      imgs_anns = json.load(f)
    # sort imgs_anns based on id
    imgs_anns=sorted(imgs_anns,key=lambda i:i['image_id'])
    imgID={item['image_id']for item in imgs_anns}
    
    for ID in imgID:

      record={}
      anns=[d for d in imgs_anns if d['image_id']==ID]
      filename = os.path.join(set_name, anns[0]['file_name'])
       

      width,height = Image.open(filename,mode="r").size
      record["file_name"] = filename
      record["image_id"] = ID
      record["height"] = height
      record["width"] = width
      objs=[]
      
      # combine all instance from each image
      for indx, img in enumerate(anns): 
            obj={
                "bbox": img['bbox'],
                "bbox_mode": BoxMode.XYWH_ABS,
                "segmentation": img['segmentation'],
                "category_id": 0,
                "category_name":img['category_name'],
                "iscrowd": img['iscrowd'],
                "area":img['area'],
                }
            objs.append(obj)
      record["annotations"] =objs
      dataset.append(record)
     
  else:

    # for test images
    for filename in os.listdir(set_name):
      
      # ignore file: there is a wired file in my folder, end with '.ini'
      if not filename.endswith('.ini'):
        record={}
        file_name=os.path.join(set_name,filename)
        width,height=Image.open(file_name,mode="r").size
        record['file_name']=file_name
        record['height']=height
        record['width']=width
        record['annotations']=[]
        dataset.append(record)

  return dataset


In [8]:
'''
# Remember to add your dataset to DatasetCatalog and MetadataCatalog
# Consdier "data_detection_train" and "data_detection_test" for registration
# You can also add an optional "data_detection_val" for your validation by spliting the training data
# TODO: approx 5 lines
'''
# refer from: https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5#scrollTo=PIbAM2pv-urF
for d in ["train","test"]:
    DatasetCatalog.register("data_detection_" + d, lambda d=d: get_detection_data('data/'+d))
    MetadataCatalog.get("data_detection_" + d).set(thing_classes=["plane"])
data_metadata = MetadataCatalog.get("data_detection_train")

In [ ]:
'''
# Visualize some samples using Visualizer to make sure that the function works correctly
# TODO: approx 5 lines
'''

#refer from: https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5#scrollTo=PIbAM2pv-urF

dataset_dicts = get_detection_data("data/train")
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=data_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])

### Set Configs

In [12]:
'''
# Set the configs for the detection part in here.
# TODO: approx 15 lines
'''
'''
# Set the configs for the detection part in here.
# TODO: approx 15 lines
'''
cfg = get_cfg()
cfg.OUTPUT_DIR = "{}/output/".format(BASE_DIR)
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("data_detection_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 500    # 500 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512  # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (plane). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)


### Training

In [ ]:
'''
# Create a DefaultTrainer using the above config and train the model
# TODO: approx 5 lines
'''
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

### Evaluation and Visualization

In [14]:
'''
# After training the model, you need to update cfg.MODEL.WEIGHTS
# Define a DefaultPredictor
'''
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
'''
# Visualize the output for 3 random test samples
# TODO: approx 10 lines
'''
def visualization_part1():
  dataset_dicts = get_detection_data("data/test")
  for d in random.sample(dataset_dicts, 3): 
      #d=dataset_dicts[0]  
      im = cv2.imread(d["file_name"])
      outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
      v = Visualizer(im[:, :, ::-1],
                    metadata=data_metadata, 
                    scale=0.5, 
                    #instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
      )
      
      out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
      cv2_imshow(out.get_image()[:, :, ::-1])

visualization_part1()

In [ ]:
'''
# Use COCOEvaluator and build_detection_train_loader
# You can save the output predictions using inference_on_dataset
# TODO: approx 5 lines
'''
# eval train
evaluator = COCOEvaluator("data_detection_train", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "data_detection_train")
print(inference_on_dataset(trainer.model, val_loader, evaluator))


### Improvements

For this part, you can bring any improvement which you have by adding new input parameters to the previous functions or defining new functions and variables.

In [54]:
'''
# Bring any changes and updates regarding the improvement in here
'''
cfg = get_cfg()
cfg.OUTPUT_DIR = "{}/output/".format(BASE_DIR)
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("data_detection_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.005  # pick a good LR
cfg.SOLVER.MAX_ITER = 400    # 500 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512  # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only 

In [ ]:
'''
# Create a DefaultTrainer using the above config and train the model
# TODO: approx 5 lines
'''
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [56]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
evaluator = COCOEvaluator("data_detection_train", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "data_detection_train")
print(inference_on_dataset(trainer.model, val_loader, evaluator))

## Part 2: Semantic Segmentation

### Data Loader

In [21]:
save_info=[]
save_id=[]
train_dicts = get_detection_data("data/train")
def crop_all_box(dataset_dicts):

  for j in range(len(dataset_dicts)):

    data=dataset_dicts[j]
    img=Image.open(data['file_name'])

    for i in range(len(data['annotations'])):
      obj=data["annotations"][i]
      
      # cropped images
      obj_img=img.crop((obj['bbox'][0],obj['bbox'][1],obj['bbox'][0]+obj['bbox'][2],obj['bbox'][1]+obj['bbox'][3]))
      obj_img=obj_img.resize((128,128))
      save_info.append(obj_img)
      save_id.append((j,i))

 
crop_all_box(train_dicts)

In [22]:
'''
# Write a function that returns the cropped image and corresponding mask regarding the target bounding box
# idx is the index of the target bbox in the data
# high-resolution image could be passed or could be load from data['file_name']
# You can use the mask attribute of detectron2.utils.visualizer.GenericMask 
#     to convert the segmentation annotations to binary masks
# TODO: approx 10 lines
'''
from detectron2.utils.visualizer import GenericMask

def normalize_data(obj):
  x_min=min(obj["segmentation"][0][0::2])
  x_max=max(obj["segmentation"][0][0::2])
  y_min=min(obj["segmentation"][0][1::2])
  y_max=max(obj["segmentation"][0][1::2])
  
  # create mask
  for i in range(len(obj["segmentation"][0])):
    val=obj["segmentation"][0][i]
    if i%2==0:
      obj["segmentation"][0][i]=((val-x_min)*128)/(x_max-x_min)
    else:
      obj["segmentation"][0][i]=((val-y_min)*128)/(y_max-y_min)
  return obj["segmentation"]


def get_instance_sample(data, idx, img=None):
  
  index=save_id.index((idx[0],idx[1]))
  obj_img=save_info[index]
  
  obj=data["annotations"][idx[1]]
  obj['segmentation']=normalize_data(obj)
  obj_mask=GenericMask(obj['segmentation'],128,128).mask

  return obj_img, obj_mask

In [28]:
'''
# We have provided a template data loader for your segmentation training
# You need to complete the __getitem__() function before running the code
# You may also need to add data augmentation or normalization in here
'''

class PlaneDataset(Dataset):
  def __init__(self, set_name, data_list):
      self.transforms = transforms.Compose([
        
          transforms.ToTensor(), # Converting the image to tensor and change the image format (Channels-Last => Channels-First)
          transforms.Normalize(0,1)
      ])
      self.set_name = set_name
      self.data = data_list
      self.instance_map = []
      for i, d in enumerate(self.data):
        for j in range(len(d['annotations'])):
          self.instance_map.append([i,j])

  '''
  # you can change the value of length to a small number like 10 for debugging of your training procedure and overfeating
  # make sure to use the correct length for the final training
  '''
  def __len__(self):
    return len(self.instance_map)

  def numpy_to_tensor(self, img, mask):
    if self.transforms is not None:
        img = self.transforms(img)
    img = torch.tensor(img, dtype=torch.float)
    mask = torch.tensor(mask, dtype=torch.float)
    return img, mask

  '''
  # Complete this part by using get_instance_sample function
  # make sure to resize the img and mask to a fixed size (for example 128*128)
  # you can use "interpolate" function of pytorch or "numpy.resize"
  # TODO: 5 lines
  '''
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
        idx = idx.tolist()
    idx = self.instance_map[idx]
    data = self.data[idx[0]]

    img,mask=get_instance_sample(data,idx)
    img=np.asarray(img)
    img=np.resize(img,(128,128,3))
    mask=np.resize(mask,(1,128,128))
    img,mask=self.numpy_to_tensor(img,mask)
    return img, mask

def get_plane_dataset(set_name='train', batch_size=2):
    my_data_list = DatasetCatalog.get("data_detection_{}".format(set_name))
    dataset = PlaneDataset(set_name, my_data_list)
    loader = DataLoader(dataset, batch_size=batch_size, num_workers=4,
                                              pin_memory=True, shuffle=True)
    return loader, dataset

### Network

In [29]:
'''
# convolution module as a template layer consists of conv2d layer, batch normalization, and relu activation
'''
class conv(nn.Module):
    def __init__(self, in_ch, out_ch, activation=True):
        super(conv, self).__init__()
        if(activation):
          self.layer = nn.Sequential(
              
             nn.Conv2d(in_ch,64, 3, padding=1),
             nn.BatchNorm2d(64),
             nn.LeakyReLU(inplace=True),
          
              

             nn.Conv2d(64, 64, 3, padding=1),
             nn.BatchNorm2d(64),
             nn.LeakyReLU(inplace=True), 
             nn.Conv2d(64, 128, 3, padding=1),
             nn.BatchNorm2d(128),
             nn.LeakyReLU(inplace=True), 

             nn.Conv2d(128, 64, 3, padding=1),
             nn.BatchNorm2d(64),
             nn.LeakyReLU(inplace=True), 

            

             nn.Conv2d(64, out_ch, 3, padding=1),
             nn.BatchNorm2d(out_ch),
             nn.LeakyReLU(inplace=True), 
        
          
          
          )
        else:
          self.layer = nn.Sequential(
             nn.Conv2d(in_ch, out_ch, 3, padding=1),
             nn.BatchNorm2d(out_ch)   
             )

    def forward(self, x):
        x = self.layer(x)
        return x

'''
# downsampling module equal to a conv module followed by a max-pool layer
'''
class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.layer = nn.Sequential(
            conv(in_ch, out_ch),
            nn.BatchNorm2d(out_ch),
            nn.MaxPool2d(2),
            nn.LeakyReLU(inplace=True),
            )

    def forward(self, x):
        x = self.layer(x)
        return x

'''
# upsampling module equal to a upsample function followed by a conv module
'''
class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=False):
        super(up, self).__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch, in_ch, 2, stride=2)

        self.conv = conv(in_ch, out_ch)

    def forward(self, x):
        y = self.up(x)
        y = self.conv(y)
        return y

'''
# the main model which you need to complete by using above modules.
# you can also modify the above modules in order to improve your results.
'''
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        
        # Encoder
        
        self.input_conv = conv(3, 4)
        self.down = down(4, 8)
        
        # Decoder
        
        self.up = up(8, 4)
        self.output_conv = conv(4, 1, False) # ReLu activation is removed to keep the logits for the loss function
        

    def forward(self, input):
      y = self.input_conv(input)
      y = self.down(y)
      y = self.up(y)
      output = self.output_conv(y)
      return output

### Training

In [ ]:
'''
# The following is a basic training procedure to train the network
# You need to update the code to get the best performance
# TODO: approx ? lines
'''

# Set the hyperparameters
num_epochs = 25
batch_size = 4
learning_rate = 0.03
weight_decay = 1e-5

model = MyModel() # initialize the model
model = model.cuda() # move the model to GPU
loader, _ = get_plane_dataset('train', batch_size) # initialize data_loader
crit = nn.BCEWithLogitsLoss() # Define the loss function
optim = torch.optim.SGD(model.parameters(),lr=learning_rate,weight_decay=weight_decay) # Initialize the optimizer as SGD

# start the training procedure
for epoch in range(num_epochs):
  total_loss = 0

  for (img, mask) in tqdm(loader):
    img = torch.tensor(img, device=torch.device('cuda'), requires_grad = True)
    mask = torch.tensor(mask, device=torch.device('cuda'), requires_grad = True)
    pred = model(img)
    loss = crit(pred, mask)
    optim.zero_grad()
    loss.backward()
    optim.step()
    total_loss += loss.cpu().data
  print("Epoch: {}, Loss: {}".format(epoch, total_loss/len(loader)))
  torch.save(model.state_dict(), '{}/output/{}_segmentation_model.pth'.format(BASE_DIR, epoch))

'''
# Saving the final model
'''
torch.save(model.state_dict(), '{}/output/final_segmentation_model.pth'.format(BASE_DIR))


### Evaluation and Visualization

In [ ]:
'''
# Before starting the evaluation, you need to set the model mode to eval
# You may load the trained model again, in case if you want to continue your code later
# TODO: approx 15 lines
'''
batch_size = 8
model = MyModel().cuda()
model.load_state_dict(torch.load('{}/output/final_segmentation_model.pth'.format(BASE_DIR)))
model = model.eval() # chaning the model to evaluation mode will fix the bachnorm layers
loader, dataset = get_plane_dataset('train', batch_size)

total_iou = 0
threshold=0.55
m=nn.Sigmoid()
num=0
for (img, mask) in tqdm(loader):
  with torch.no_grad():
    img = img.cuda()
    mask = mask.cuda()
    mask = torch.unsqueeze(mask,1)
    pred = model(img)

    '''
    ## Complete the code by obtaining the IoU for each img and print the final Mean IoU
    '''
    pred=m(pred)
    pred=pred.cpu().numpy()
    mask=mask.cpu().numpy()
    for img_pred in range(len(pred)):
      
      pred[img_pred][0][pred[img_pred][0]>threshold]=1
      pred[img_pred][0][pred[img_pred][0]<threshold]=0
   
      y_true=np.array(mask[img_pred][0][0]).flatten()
      y_pred=np.array(pred[img_pred][0]).flatten()
      total_iou+=jaccard_score(y_true,y_pred.round())
    num+=len(pred)
    
print("\n #images: {}, Mean IoU: {}".format(num,total_iou/num))

In [22]:
 
def visualize_test(pred_boxes,image,flag=1,idx=0):
  
  m=nn.Sigmoid()
  threshold=0.55
  x,y,x1,y1=pred_boxes[idx].tensor.cpu().numpy()[0]

  image=image.crop((x,y,x1,y1))
  image=image.resize((128,128))
  image=np.asarray(image)
  image=np.resize(image,(128,128,3))
  
  pro_data=transforms.ToTensor()(image)
  pro_data=transforms.Normalize(0,1)(pro_data)
  pro_data=pro_data.reshape(1,3,128,128)
  
  with torch.no_grad():
    pred=model(pro_data.cuda())
    pred=m(pred)
    pred=pred.cpu().numpy()
    pred[0][0][pred[0][0]>threshold]=1
    pred[0][0][pred[0][0]<threshold]=0
    if flag==1:
      cv2_imshow(image*255)
      cv2_imshow(pred[0][0]*255)
    else:
      return pred,x,x1,y,y1 

In [20]:
'''
# Visualize 3 sample outputs
# TODO: approx 5 lines
'''
def visualization_part2():
  test_dicts = get_detection_data("data/test")
  for d in random.sample(test_dicts, 3): 
    arr_img = cv2.imread(d["file_name"])
    outputs = predictor(arr_img)
    pred_boxes=outputs['instances'].get_fields()['pred_boxes']
    image=Image.open(d["file_name"])
    visualize_test(pred_boxes,image)

visualization_part2()

## Part 3: Instance Segmentation

In this part, you need to obtain the instance segmentation results for the test data by using the trained segmentation model in the previous part and the detection model in Part 1.

In [24]:
def back_to(data,idx):
  obj=data["annotations"][idx[1]]
  mask=GenericMask(obj['segmentation'],data['height'],data['width']).mask
  mask=mask*(idx[1]+1)
  return mask
 

### Get Prediction

In [25]:
'''
# Define a new function to obtain the prediction mask by passing a sample data
# For this part, you need to use all the previous parts (predictor, get_instance_sample, data preprocessings, etc)
# TODO: approx 35 lines
'''
def get_prediction_mask(data,ID):
  
  # read images
  gt_mask=0
  img=Image.open(data['file_name'])
  if len(img.size)!=3:
    img=img.convert('RGB')
 
  pass_image=img
  width,height=img.size
  img= np.asarray(img)
   
  # pred_mask
  outputs = predictor(img)
  whole=np.zeros((height,width))

  pred_boxes=outputs['instances'].get_fields()['pred_boxes']
  for idx in range(len(pred_boxes)):
    pred,x,x1,y,y1=visualize_test(pred_boxes,pass_image,0,idx)
    pred=pred*(idx+1)
    x=int(x.round())
    x1=int(x1.round())
    y=int(y.round())
    y1=int(y1.round())

    w=x1-x
    h=y1-y
    pred=pred[0][0]
    pred_mask=Image.fromarray(np.uint8(pred))
    pred_mask=pred_mask.resize((w,h))
    whole[y:y1,x:x1]=pred_mask
  pred_mask=whole
  
  
  # get gt_mask (don't execute when generating outputs)
  if len(data['annotations'])!=0:
    gt_mask=back_to(data,[ID,0])
    for i in range(1,len(data['annotations'])):
        gt_mask= back_to(data,[ID,i])+gt_mask

    if gt_mask.dtype=='uint16':
      gt_mask=np.uint8(gt_mask)
    gt_mask=torch.tensor(gt_mask, device=torch.device('cuda'))

   
  img=torch.tensor(img, device=torch.device('cuda'))
  pred_mask=torch.tensor(pred_mask, device=torch.device('cuda'))
   
  
 
  return img, gt_mask, pred_mask
  

### Visualization and Submission

In [ ]:
'''
# Visualise the output prediction as well as the GT Mask and Input image for a sample input
# TODO: approx 10 lines
'''
def visualization_whole():
  test_dicts = get_detection_data("data/test")
  random_list=np.random.randint(0,len(test_dicts)-1,size=1)
  for i in random_list:
    img,gt_mask,pred_mask=get_prediction_mask(test_dicts[i],i)
    
    img=cv2.cvtColor(img.cuda().cpu().numpy(),cv2.COLOR_BGR2GRAY)
    pred_mask=cv2.normalize(pred_mask.cuda().cpu().numpy(),None,0,255,cv2.NORM_MINMAX,dtype=cv2.CV_8U)
    merge=np.hstack((img,pred_mask))
    cv2_imshow(cv2.resize(merge,(800,600)))
    
visualization_whole()





In [41]:
'''
# ref: https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
# https://www.kaggle.com/c/airbus-ship-detection/overview/evaluation
'''
def rle_encoding(x):
    '''
    x: numpy array, 1 - mask, 0 - background
    Returns run length as list
    '''
    dots = torch.where(torch.flatten(x.long())==1)[0]
    if(len(dots)==0):
      return []
    inds = torch.where(dots[1:]!=dots[:-1]+1)[0]+1
    inds = torch.cat((torch.tensor([0], device=torch.device('cuda'), dtype=torch.long), inds))
    tmpdots = dots[inds]
    inds = torch.cat((inds, torch.tensor([len(dots)], device=torch.device('cuda'))))
    inds = inds[1:] - inds[:-1]
    runs = torch.cat((tmpdots, inds)).reshape((2,-1))
    runs = torch.flatten(torch.transpose(runs, 0, 1)).cpu().data.numpy()
    return ' '.join([str(i) for i in runs])

In [48]:
'''
# You need to upload the csv file on kaggle
# The speed of your code in the previous parts highly affects the running time of this part
'''

preddic = {"ImageId": [], "EncodedPixels": []}

'''
# Writing the predictions of the training set
'''
my_data_list = DatasetCatalog.get("data_detection_{}".format('train'))
for i in tqdm(range(len(my_data_list)), position=0, leave=True):
  sample = my_data_list[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask = get_prediction_mask(sample,i)
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for index in inds:
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index)
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)

'''
# Writing the predictions of the test set
'''

my_data_list = DatasetCatalog.get("data_detection_{}".format('test'))
for i in tqdm(range(len(my_data_list)), position=0, leave=True):
  sample = my_data_list[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask = get_prediction_mask(sample,i)
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for j, index in enumerate(inds):
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index).double()
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)

pred_file = open("{}/pred.csv".format(BASE_DIR), 'w')
pd.DataFrame(preddic).to_csv(pred_file, index=False)
pred_file.close()


## Part 4: Mask R-CNN

For this part you need to follow a same procedure to part 2 with the configs of Mask R-CNN, other parts are generally the same as part 2.

### Data Loader

In [9]:
dataset_dicts = get_detection_data("data/test")

### Network

In [10]:
cfg = get_cfg()
cfg.OUTPUT_DIR = "{}/output/".format(BASE_DIR)
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("data_detection_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.005  # pick a good LR
cfg.SOLVER.MAX_ITER = 400    # 500 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512  # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only 

### Training

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train() 

In [12]:
# update weight and set predictor
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

### Evaluation and Visualization

In [ ]:
'''
# Use COCOEvaluator and build_detection_train_loader
# You can save the output predictions using inference_on_dataset
# TODO: approx 5 lines
'''
# eval train
evaluator = COCOEvaluator("data_detection_train", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "data_detection_train")
print(inference_on_dataset(trainer.model, val_loader, evaluator))


In [ ]:
visualization_part1()